In [7]:
import intake
import warnings 
import hvplot.pandas
import hvplot.xarray
import cartopy.crs as ccrs
from holoviews.operation.datashader import rasterize
import geoviews as gv

**Open the NWM intake catalog**

In [8]:

catalog = intake.open_catalog('s3://nextgen-dmac-cloud-ingest/nwm_ll/nwm_intake.yml',storage_options={'anon':True})
(list(catalog))

['NWM_Best_CONUS_Short_Range',
 'NWM_Best_CONUS_Medium_Range',
 'NWM_Best_Alaska_Short_Range',
 'NWM_Best_Alaska_Medium_Range',
 'NWM_Best_Hawaii_Short_Range',
 'NWM_Best_PuertoRico_Short_Range']


&nbsp;

**Select the catalog entry to open**

In [9]:
cat = catalog['NWM_Best_CONUS_Short_Range']
ds = cat.to_dask()
ds

<xarray.Dataset>
Dimensions:         (feature_id: 2776734, time: 327, reference_time: 1)
Coordinates:
  * feature_id      (feature_id) int64 101 179 181 ... 1180001803 1180001804
    latitude        (feature_id) float32 dask.array<chunksize=(2776734,), meta=np.ndarray>
    longitude       (feature_id) float32 dask.array<chunksize=(2776734,), meta=np.ndarray>
  * reference_time  (reference_time) datetime64[ns] 2024-02-15T14:00:00
  * time            (time) datetime64[ns] 2024-02-15T15:00:00 ... 2024-02-29T...
Data variables:
    altitude        (feature_id) float32 dask.array<chunksize=(2776734,), meta=np.ndarray>
    crs             (time) object dask.array<chunksize=(1,), meta=np.ndarray>
    from_link_id    (feature_id) float64 dask.array<chunksize=(2776734,), meta=np.ndarray>
    link_id         (feature_id) float64 dask.array<chunksize=(2776734,), meta=np.ndarray>
    nudge           (time, feature_id) float64 dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    qBtmVertRunoff  (time, feature_id) float64 dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    qBucket         (time, feature_id) float64 dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    qSfcLatRunoff   (time, feature_id) float64 dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    streamflow      (time, feature_id) float64 dask.array<chunksize=(1, 2776734), meta=np.ndarray>
    terminal_node   (feature_id) float64 dask.array<chunksize=(2776734,), meta=np.ndarray>
    to_link_id      (feature_id) float64 dask.array<chunksize=(2776734,), meta=np.ndarray>
    velocity        (time, feature_id) float64 dask.array<chunksize=(1, 2776734), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                CF-1.6
    NWM_version_number:         v3.0
    TITLE:                      OUTPUT FROM NWM v3.0
    cdm_datatype:               Station
    code_version:               v5.3.0-alpha1
    dev:                        dev_ prefix indicates development/internal me...
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    2024-02-15_15:00:00
    model_total_valid_times:    18
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1




&nbsp;
**Set the spatial bounds and time to extract**

In [10]:
lon_min= -180.
lat_min=24
lon_max=-60
lat_max=180
time_s='2024-02-26T18:00'

&nbsp;

**Extract the data**

In [11]:
%%time
ids = ((ds.latitude > lat_min) & (ds.latitude < lat_max) & (ds.longitude > lon_min) & (ds.longitude < lon_max)).compute()
s_ds = ds.sel(time=time_s).isel(feature_id=ids)

stream_ds = s_ds['streamflow']

CPU times: user 528 ms, sys: 551 ms, total: 1.08 s
Wall time: 1.53 s


&nbsp;

**Plot the extracted data**

In [12]:


df=stream_ds.to_pandas().to_frame()
df = df.assign(longitude = stream_ds.longitude)
df =df.assign(latitude=stream_ds.latitude)
df = df.assign(fid=s_ds.feature_id)
df = df.assign(fid=s_ds.link_id)
df.rename(columns={0:"streamflow"},inplace=True)

p = df.hvplot.points('longitude', 'latitude', crs=ccrs.PlateCarree(),
                     c='streamflow',size=14)
g = rasterize(p, aggregator='mean', x_sampling=0.001, y_sampling=0.001).opts(tools=['hover'], 
                 aspect='equal',colorbar=True,logz=True, cmap='viridis', clim=(1e-2, 2000), width=500,height=400 )

g  * gv.tile_sources.OSM

:DynamicMap   []
   :Overlay
      .Image.I :Image   [longitude,latitude]   (longitude_latitude streamflow)
      .WMTS.I  :WMTS   [Longitude,Latitude]